# **Adding Observability with LangSmith**

## **What's Covered?**
1. Introduction to LangSmith
    - What is LangSmith?
    - What is Observability and Telemetry Data?
    - Important Terminology
    - What does LangSmith Records?
    - Setting up LangSmith
2. Example Implementation
    - Step 1: Add the required LangSmith environment variables
    - Step 2: Create a Chain
    - Step 3: Invoking the Chain
    - Step 4: Adding the tags and metadata
3. Example Implementation 2

## **Introduction to LangSmith**

### **What is LangSmith?**
Provides observability & evaluation which helps to debug, test and monitor AI systems/workflows. Eg: Identify why your production workflow is taking more time (i.e. latency), measure the cost, token usage, hallucination, etc...

### **What is Observability?**
Ability to understand a system's internal state by analyzing its external outputs, primarily through telemetry data (i.e. logs, metrics and traces)
- **Logs:** Chronological records of events, actions and messages from the system
- **Metrics:** Numerical measurement over time eg: CPU usage, number of requests, latency, etc...
- **Traces:** Records the journey of a single request as it moves across different services in a distributed system, showing timing and flow.

### **Important Terminology**
- **Run:** Each step within a trace is represented by a run. A run is a span representing a single unit of work or operation within your LLM application. This could be anything from a single call to an LLM or chain, to a prompt formatting call, to a runnable lambda invocation.
- **Thread:** A thread is a sequence of traces representing a single conversation. Many LLM applications have a chatbot-like interface in which the user and the LLM application engage in a multi-turn conversation. Each turn in the conversation is represented as its own trace, but these traces are linked together by being part of the same thread.
- **Projects:** A project is a collection of traces. You can think of a project as a container for all the traces that are related to a single application or service. You can have multiple projects, and each project can have multiple traces.
- **Feedback:** Feedback allows you to score an individual run based on certain criteria. Each feedback entry consists of a feedback tag and feedback score, and is bound to a run by a unique run ID. Feedback can be continuous or discrete (categorical), and you can reuse feedback tags across different runs within an organization.
- **Tags:** Tags are collections of strings that can be attached to runs. You can use tags to do the following in the LangSmith UI: Categorize runs for easier search, Filter runs and Group runs together for analysis.

### **What does LangSmith records?**
1. Inputs and Outputs
2. Token Usage
3. Cost
4. Latency
5. Errors
6. Intermediate steps
7. Tags
8. Metadata
9. Feedback

[Click Here](https://docs.langchain.com/langsmith/observability-concepts) to read the official LangSmith documentation.

### **Setting up LangSmith**

You need to setup the following environment variables initially:
1. LANGSMITH_TRACING=true
2. LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
3. LANGSMITH_API_KEY=`"<api-key>"`
4. LANGSMITH_PROJECT=`"<project-name>"`

## **Example Implementation**

### **Step 1: Add the required LangSmith environment variables**

In [1]:
import os

# Setup LANGSMITH API Key
f = open('keys/.langsmith_api_key.txt')
LANGSMITH_API_KEY = f.read()

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = LANGSMITH_API_KEY
os.environ["LANGSMITH_PROJECT"] = "my-simple-chain-project"

### **Step 2: Create a Chain**

In [2]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate(
    messages=
    [
        ("system", "You are a polite assistant."), 
        ("human", "{human_input}")
    ]
)

/Users/kanavbansal/Developer/.env_langchain/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain_openai import ChatOpenAI

# Setup API Key
f = open('keys/.openai_api_key.txt')
OPENAI_API_KEY = f.read()

openai_chat_model = ChatOpenAI(api_key=OPENAI_API_KEY, 
                               model="gpt-4o-mini", 
                               temperature=1)

In [4]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [5]:
chain = chat_template | openai_chat_model | output_parser

### **Step 3: Invoking the Chain**

**Note: By default, LangSmith is going to track all the runs which are called using .invoke() method.**

<img src="images/project_dashboard.png">

In [6]:
chain.invoke({"human_input" : "Hello!"})

'Hello! How can I assist you today?'

### **Step 4: Adding the tags and metadata**

<img src="images/run_openai.png">

In [7]:
# Configuring the Run Name, Tags and Metadata
openai_config = {
    "run_name": "openai chain",
    "tags": ["openai", "gpt-4o-mini"],
    "metadata": {"inference": "openai", "llm": "gpt-4o-mini"}
}

chain.invoke({"human_input" : "Can you explain in 4-5 words what LangChain and LangGraph does?"}, config=openai_config)

'LangChain facilitates LLM application development. LangGraph enables graph-based data processing.'

## **Example Implementation 2**

<img src="images/run_groq.png">

In [8]:
# Setup API Key
from langchain_groq import ChatGroq

f = open('keys/.groq_api_key.txt')
GROQ_API_KEY = f.read()

# Pass the standard parameters during initialization
groq_chat_model = ChatGroq(api_key=GROQ_API_KEY, 
                           model="openai/gpt-oss-20b", 
                           temperature=1)

In [9]:
chain_2 = chat_template | groq_chat_model | output_parser

In [10]:
# Configuring the Run Name, Tags and Metadata
openai_config = {
    "run_name": "groq chain",
    "tags": ["groq", "gpt-oss-20b"],
    "metadata": {"inference": "groq", "llm": "gpt-oss-20b"}
}

chain_2.invoke({"human_input" : "Can you explain in 4-5 words what LangChain and LangGraph does?"}, config=openai_config)

'- **LangChain:** “Enables building AI apps.”  \n- **LangGraph:** “Builds knowledge‑graph workflows.”'